In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import norm
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten, Attention
from tensorflow.keras.models import Model
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def get_stock_data(ticker, start='2024-01-01', end='2025-02-25'):
    stock = yf.download(ticker, start=start, end=end)['Close']
    return stock.pct_change().dropna()

In [ ]:
# Kaggle function

def make_model():
    inp = Input(shape=(128, 10))
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = Attention(128)(x)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64, activation="relu")(x)
    x = Dense(9, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# My function
def make_model():
    # 1. Input layer for stock data (sequence_length, features)
    inp = Input(shape=(128, 10))  # Example: 128 timesteps, 10 features (adjust as needed)

    # 2. Bidirectional LSTM layers for sequence learning
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)

    # 3. Dropout to prevent overfitting
    x = Dropout(0.2)(x)

    # 4. 1D Convolutional layer for feature extraction
    x = Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(x)

    # 5. MaxPooling to downsample
    x = MaxPooling1D(pool_size=2)(x)

    # 6. Flatten layer to transition to fully connected layers
    x = Flatten()(x)

    # 7. Fully connected layers for high-level learning
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)

    # 8. Output layer - Single neuron with tanh activation to produce values between -1 and 1
    output = Dense(1, activation="tanh")(x)

    # 9. Compile model with Adam optimizer
    model = Model(inputs=inp, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    return model

# Create the model
model = make_model()
model.summary()
